# HTML operation
> HTML operation in jupyter notebook, easier life with visualization

In [1]:
# default_exp html

In [2]:
# export
from IPython.display import HTML

class DOM:
    def __init__(self,txt,tag,kwargs = dict()):
        self.txt = txt
        self.tag = str(tag).lower()
        self.attrs = kwargs
        self.refresh_attr()
        
    @staticmethod
    def extend(text,tag,**kwargs):
        attributes =(" ".join(f'{k}="{v}"' for k,v in kwargs.items()))
        attributes=" "+attributes if attributes else attributes
        start = f"<{tag}{attributes}>"
        inner = f"{text}"
        end = f"</{tag}>"
        text = f"{start}{inner}{end}"
        return start,inner,end
    
    def refresh_attr(self):
        self.start,self.inner,self.end = self.extend(self.txt,self.tag,**self.attrs)
    
    def __mul__(self,new_tag):
        assert type(new_tag)==str
        return DOM(self.text,new_tag)
    
    def __add__(self,dom):
        return self.text+dom.text
    
    def __repr__(self):
        return f"{self.start}{self.inner}{self.end}"
    
    def __getitem__(self,k):
        return self.attrs[k]
    
    def __setitem__(self,k,v):
        self.update({k,v})
        
    def __call__(self):
        self.display()
    
    @property
    def text(self):
        return str(self)
    
    def append(self,subdom):
        self.inner = self.inner+str(subdom)
        return self
    
    def update(self,dict_):
        self.attrs.update(dict_)
        self.refresh_attr()
        return self
    
    def display(self):
        display(HTML(self.text))

## Create HTML tag

In [3]:
btn = DOM("Hello",tag="div",kwargs = {"class":"btn btn-sm btn-danger","id":"test_btn_001"})

btn

<div class="btn btn-sm btn-danger" id="test_btn_001">Hello</div>

Show HTML raw text

In [4]:
btn.text

'<div class="btn btn-sm btn-danger" id="test_btn_001">Hello</div>'

Display in HTML

In [5]:
btn()

## Extend tag

In [6]:
DOM("a","div")*"span"

<span><div>a</div></span>

## DOM operation

In [17]:
ul = DOM("","ul")
ul

<ul class="list-group"></ul>

In [18]:
ul.update({"class":"list-group"})

<ul class="list-group"></ul>

In [19]:
for i in range(5):
    ul.append(DOM(f"List item{i}", "li",{"class":"list-group-item paint-red"}))

In [20]:
ul()

In [3]:
# export 
def deeper(x):
    if type(x) in [list, set, tuple]:
        return list_group(x)
    if type(x) == dict:
        return list_group_kv(x)
    return x

def list_group(iterable):
    ul = DOM("","ul",{"class":"list-group"})
    for i in iterable:
        li = DOM(deeper(i),"li",{"class":"list-group-item"})
        ul.append(li)
    return ul

In [9]:
# export
import math
def col_sm(iterable,portions = None,):
    if portions == None:
        portions = [math.floor(12/len(iterable)),]* len(iterable)
    row = DOM("","div",{"class":"row"})
    for i,p in zip(iterable,portions):
        row.append(DOM(i,"div",{"class":f"col-sm-{p}"}))
    return row
        

In [14]:
col_sm(["foo","bar","return"])()

In [15]:
col_sm(["foo","bar","return"],[2,5,5])()

In [5]:
# export
def list_group_kv(data):
    result = []
    for k,v in data.items():
        row = DOM("","div",{"class":"row"})
        row.append(DOM(f"{k}","strong",{"class":"col-sm-5"}))\
            .append(DOM(f"{deeper(v)}","span",{"class":"col-sm-7"}))
        result.append(row)
    return list_group(result)

## Example: Running JavaScript

In [21]:
DOM("alert(123)","script")

<script class="list-group">alert(123)</script>

execute js script

In [ ]:
DOM("alert(123)","script")()

Longer script

In [23]:
paint_red = """
console.log(document.querySelectorAll(".paint-red"))
document.querySelectorAll(".paint-red").forEach(dom=>{
dom.className=dom.className+" text-danger"
})
"""

DOM(paint_red,"script")()

### A class designed for running javascript

In [46]:
# export
def JS(code):
    DOM(code,"script",)()
    
def JS_file(path):
    """
    load javascript file 
    """
    with open(path,"r") as f:
        DOM(f.read(),"script")()